# YOLOv5 Unified Inference — Google Colab
Runs **PyTorch** and **ONNX** inference side-by-side on COCO val2017.

Features:
- Bounding-box images saved to `results/bbox_outputs/`
- Class labels + confidence scores
- MSE baseline comparison (PyTorch ↔ ONNX raw logits)
- mAP@0.5 accuracy metrics for both backends

**Set your GitHub PAT below, then Run All.**

In [ ]:
# ── 0. Configuration ───────────────────────────────────────────────────────
GITHUB_TOKEN = "YOUR_GITHUB_PAT_HERE"   # <-- paste your token
REPO_OWNER   = "ARNiteshKumar"
REPO_NAME    = "MAGIC-Cluster_YoLoV5"
BRANCH       = "claude/yolov5-coco-validation-oToNk"

# Model weights — set to None to skip that backend
PT_WEIGHTS   = "yolov5s.pt"       # download below if missing
ONNX_WEIGHTS = "yolov5s.onnx"     # exported from PT weights below

# Evaluation settings
NUM_EVAL_IMAGES = 200    # set to 5000 for full COCO val run (slow on CPU)
SAVE_VIS        = 20     # save annotated images for first N val images
CONF_THRESH     = 0.25
IOU_THRESH      = 0.45
DEVICE          = "cpu"  # or 'cuda'

In [ ]:
# ── 1. Clone / pull the repo ───────────────────────────────────────────────
import os

REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{REPO_OWNER}/{REPO_NAME}.git"

if not os.path.isdir(REPO_NAME):
    !git clone --branch {BRANCH} {REPO_URL}
else:
    !git -C {REPO_NAME} pull origin {BRANCH}

%cd {REPO_NAME}
print("Working dir:", os.getcwd())

In [ ]:
# ── 2. Install dependencies ────────────────────────────────────────────────
!pip install -q -r requirements.txt

# Clone YOLOv5 helper repo (provides model classes)
if not os.path.isdir('yolov5'):
    !git clone -q https://github.com/ultralytics/yolov5
    !pip install -q -r yolov5/requirements.txt

In [ ]:
# ── 3. Download YOLOv5s weights + export to ONNX ──────────────────────────
import torch

# Download pretrained YOLOv5s
if not os.path.isfile(PT_WEIGHTS):
    torch.hub.download_url_to_file(
        'https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt',
        PT_WEIGHTS
    )
    print(f"Downloaded {PT_WEIGHTS}")

# Export to ONNX
if not os.path.isfile(ONNX_WEIGHTS):
    !python yolov5/export.py \
        --weights {PT_WEIGHTS} \
        --include onnx \
        --opset 17 \
        --simplify \
        --imgsz 640
    # export.py writes alongside .pt
    onnx_src = PT_WEIGHTS.replace('.pt', '.onnx')
    if os.path.isfile(onnx_src) and onnx_src != ONNX_WEIGHTS:
        os.rename(onnx_src, ONNX_WEIGHTS)
    print(f"Exported {ONNX_WEIGHTS}")

In [ ]:
# ── 4. Download COCO val2017 subset (first NUM_EVAL_IMAGES via fiftyone) ──
# Using fiftyone for fast partial download — avoids the full 1 GB zip
!pip install -q fiftyone

import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    max_samples=NUM_EVAL_IMAGES,
    dataset_name="coco-val-subset",
    overwrite=True,
)

# Symlink into expected directory layout
import shutil, pathlib

coco_img_dir  = pathlib.Path("data/coco/images/val2017")
coco_ann_dir  = pathlib.Path("data/coco/annotations")
coco_img_dir.mkdir(parents=True, exist_ok=True)
coco_ann_dir.mkdir(parents=True, exist_ok=True)

# Export annotations
dataset.export(
    export_dir="data/coco",
    dataset_type=fo.types.COCODetectionDataset,
    label_field="ground_truth",
    overwrite=True,
)

# Move images into val2017/
for fp in pathlib.Path("data/coco/data").glob("*.jpg"):
    shutil.copy(str(fp), str(coco_img_dir / fp.name))

# Rename annotation file
ann_src = pathlib.Path("data/coco/labels.json")
ann_dst = coco_ann_dir / "instances_val2017.json"
if ann_src.is_file() and not ann_dst.is_file():
    shutil.copy(str(ann_src), str(ann_dst))

print(f"COCO subset ready: {len(list(coco_img_dir.glob('*.jpg')))} images")

In [ ]:
# ── 5. Single-image quick test ─────────────────────────────────────────────
sample_img = next(pathlib.Path("data/coco/images/val2017").glob("*.jpg"))

!python src/inference/infer.py \
    --pt-weights   {PT_WEIGHTS} \
    --onnx-weights {ONNX_WEIGHTS} \
    --image        {sample_img} \
    --conf         {CONF_THRESH} \
    --iou          {IOU_THRESH} \
    --device       {DEVICE} \
    --output-dir   results/bbox_outputs

In [ ]:
# ── 6. Display bbox output images ─────────────────────────────────────────
from IPython.display import display, Image as IPImage
import pathlib

out_imgs = sorted(pathlib.Path("results/bbox_outputs").glob("*.jpg"))[:6]
for p in out_imgs:
    print(p.name)
    display(IPImage(str(p), width=640))

In [ ]:
# ── 7. Full COCO evaluation (PyTorch + ONNX mAP, MSE comparison) ──────────
!python src/inference/infer.py \
    --pt-weights       {PT_WEIGHTS} \
    --onnx-weights     {ONNX_WEIGHTS} \
    --eval \
    --coco-dir         data/coco \
    --num-eval-images  {NUM_EVAL_IMAGES} \
    --save-vis         {SAVE_VIS} \
    --conf             {CONF_THRESH} \
    --iou              {IOU_THRESH} \
    --device           {DEVICE} \
    --output-dir       results/bbox_outputs

In [ ]:
# ── 8. Show evaluation bbox samples ───────────────────────────────────────
out_imgs = sorted(pathlib.Path("results/bbox_outputs").glob("*.jpg"))[:10]
for p in out_imgs:
    print(p.name)
    display(IPImage(str(p), width=640))